In [1]:
# Standard imports
import os
from pathlib import Path
import shutil

# Imports for data handling
import numpy as np
import pandas as pd

# Imports for efficient string matching
import ahocorasick

# Imports for progress tracking
from tqdm import tqdm

# Imports for data visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns

# Imports for data processing and statistical analysis
from scipy import stats

# Imports for network analysis and adjacency matrices analysis
import networkx as nx


In [2]:
# Set working paths.
working_dir = Path("path/to/working/directory") #Change this with you working directory

DMS_files_path = working_dir/"DMS"
DMS_samples_with_subs_path = DMS_files_path/"Substitutions"
DMS_samples_with_indels_path = DMS_files_path/"Indels"

# Set nullpeptides path
nullpeptides_path = working_dir/"nullpeptides"

# Read files for analysis as pandas dataframes.
DMS_samples_with_subs_info_df = pd.read_csv(DMS_files_path/"DMS_substitutions.csv")
DMS_samples_with_indels_info_df = pd.read_csv(DMS_files_path/"DMS_indels.csv")

# Keep only Human experiments.
human_only_DMS_samples_with_subs_info_df = DMS_samples_with_subs_info_df[DMS_samples_with_subs_info_df['taxon'] == 'Human']
human_only_DMS_samples_with_indels_info_df = DMS_samples_with_indels_info_df[DMS_samples_with_indels_info_df['taxon'] == 'Human']

# Save Human only analyses to csv.
human_only_DMS_samples_with_subs_info_df.to_csv(DMS_files_path/"human_only_DMS_substitutions.csv")
human_only_DMS_samples_with_indels_info_df.to_csv(DMS_files_path/"human_only_DMS_indels.csv")

# Set paths to save human samples.
human_DMS_samples_with_subs_path = DMS_samples_with_subs_path/"human_samples"
human_DMS_samples_with_indels_path = DMS_samples_with_indels_path/"human_samples"

# Define an output directory for each mutation type.
modified_subs_path = human_DMS_samples_with_subs_path/"modified_human_samples"
modified_indels_path = human_DMS_samples_with_indels_path/"modified_human_samples"

# Specify empty lists to store the dataframe for each sample.
modified_human_DMS_samples_with_subs_list = []
modified_human_DMS_samples_with_indels_list = []

# Specify the output directories for each mutation case.
plots_directory = DMS_files_path/"Plots"
networks_directory = DMS_files_path/"Networks"

# Specify the output directories for each mutation case.
subs_output_directory = plots_directory/"Subs"
indels_output_directory = plots_directory/"Indels"

# Set the style for all plots
sns.set_style("darkgrid")

# Skip SettingWithCopyWarnings
pd.set_option('mode.chained_assignment', None)


In [7]:
def create_directory(path):
    """
    Creates a directory at the specified path if it does not already exist.
    """

    if not os.path.exists(path):
        os.makedirs(path)

def copy_human_samples(source_path, destination_path):
    """
    Copies files containing '_HUMAN_' in their name from the source path to the destination path.
    """
    
    for filename in os.listdir(source_path):
        if "_HUMAN_" in filename:
            source_file = os.path.join(source_path, filename)
            destination_file = os.path.join(destination_path, filename)
            shutil.copy(source_file, destination_file)

# Create paths if they don't exist
create_directory(human_DMS_samples_with_subs_path)
create_directory(human_DMS_samples_with_indels_path)
create_directory(modified_subs_path)
create_directory(modified_indels_path)
create_directory(networks_directory)
create_directory(subs_output_directory)
create_directory(indels_output_directory)

# Copy human DMS samples to their respective new folders
copy_human_samples(DMS_samples_with_subs_path, human_DMS_samples_with_subs_path)
copy_human_samples(DMS_samples_with_indels_path, human_DMS_samples_with_indels_path)

In [4]:
def add_features(DMS_info, source_path, destination_path, filename, mutation_type):
    """
    For a given DMS sample file, adds a new row with the wild type sequence and two new columns (Protein and Study Type) and saves the modified DataFrame.
    """

    file_path = os.path.join(source_path, filename)

    # Extract information based on the DMS filename
    DMS_file_info = DMS_info[DMS_info['DMS_filename'] == filename]

    if not DMS_file_info.empty:
        target_seq = DMS_file_info['target_seq'].values[0]
        DMS_binarization_cutoff = DMS_file_info['DMS_binarization_cutoff'].values[0]
        coarse_selection_type = DMS_file_info['coarse_selection_type'].values[0]
        protein_name = filename.split('_')[0]  # Assuming protein name is always the first part before the first underscore

        DMS_df = pd.read_csv(file_path)
        
        # Add new columns for protein name and study type
        DMS_df['Protein'] = protein_name
        DMS_df['Study_Type'] = coarse_selection_type
        
        # Define a new row to be added for the wild type
        if mutation_type == 'subs':
            new_row = {'mutant': 'Wild Type', 'mutated_sequence': target_seq, 'DMS_score': DMS_binarization_cutoff, 'DMS_score_bin': 1}
        elif mutation_type == 'indels':
            new_row = {'mutant': target_seq, 'DMS_score': DMS_binarization_cutoff, 'DMS_score_bin': 1}
        
        DMS_df = pd.concat([pd.DataFrame([new_row]), DMS_df], ignore_index=True)

        # Save the updated DataFrame to a new CSV file
        modified_filename = os.path.join(destination_path, f'modified_{filename}')
        DMS_df.to_csv(modified_filename, index=False)


# Process substitution samples
for filename in os.listdir(human_DMS_samples_with_subs_path):
    if filename.endswith('.csv'):
        add_features(human_only_DMS_samples_with_subs_info_df, human_DMS_samples_with_subs_path, modified_subs_path, filename, 'subs')

# Process indel samples
for filename in os.listdir(human_DMS_samples_with_indels_path):
    if filename.endswith('.csv'):
        add_features(human_only_DMS_samples_with_indels_info_df, human_DMS_samples_with_indels_path, modified_indels_path, filename, 'indels')

In [5]:
# Define a function to scale all DMS protein fitness scores to specific scale.
def scale_DMS_scores(data, score_column):
    """
    Calculate a modified Z-score for each mutation, using the DMS score of the wild type sequence instead of the mean. This centers the data around the wild type score
    and calculates the deviations from this score.
    """
    
    wild_type_score = data.loc[0, score_column]
    
    # Calculate deviations from the wild type score
    deviations = data[score_column] - wild_type_score
    
    # Square these deviations
    squared_deviations = deviations ** 2
    
    # Calculate the mean of the squared deviations
    mean_squared_deviation = squared_deviations.mean()
    
    # Calculate the "custom deviation" (similar to standard deviation but centered on the wild type score)
    custom_deviation = np.sqrt(mean_squared_deviation)
    
    # Compute modified Z-scores using the custom deviation
    data['Scaled_' + score_column] = deviations / custom_deviation

    return data, wild_type_score

# Define functions to process DMS sample files
def process_DMS_sample(file_path, score_column, drop_columns, rename_columns=None):
    """
    Processes a DMS sample file by scaling scores, classifying sequences, and dropping unnecessary columns.
    """

    try:
        dms_sample_df = pd.read_csv(file_path, encoding='utf-8')
        dms_sample_df, cutoff_value = scale_DMS_scores(dms_sample_df, score_column)
        dms_sample_df['State'] = dms_sample_df[score_column].apply(lambda x: 'Benign' if x >= cutoff_value else 'Pathogenic')
        if rename_columns:
            dms_sample_df = dms_sample_df.rename(columns=rename_columns)
        dms_sample_df = dms_sample_df.drop(columns=drop_columns)
        dms_sample_df = dms_sample_df.iloc[1:]  # Drop the row with the wild type sequence
        return dms_sample_df
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def process_directory(directory_path, score_column, drop_columns, rename_columns=None):
    """
    Processes all DMS sample files in a directory by scaling scores, classifying sequences, and dropping unnecessary columns.
    """

    processed_samples = []
    for filename in filter(lambda x: x.endswith('.csv'), os.listdir(directory_path)):
        file_path = os.path.join(directory_path, filename)
        processed_df = process_DMS_sample(file_path, score_column, drop_columns, rename_columns)
        if processed_df is not None:
            processed_samples.append(processed_df)
    return processed_samples

# Process directories and update lists
modified_human_DMS_samples_with_subs_list = process_directory(
    modified_subs_path, 
    'DMS_score', 
    drop_columns=['mutant', 'DMS_score_bin'])

modified_human_DMS_samples_with_indels_list = process_directory(
    modified_indels_path,
    'DMS_score',
    drop_columns=['DMS_score_bin'],
    rename_columns={'mutant': 'mutated_sequence'})

In [8]:
# Define a function to plot the results as histograms so that to study the distribution of the scaled DMS scores for each study.
def plot_histogram(df, index, output_directory, state_column='State', score_column='DMS_score', scaled_score_column='Scaled_DMS_score'):
    plt.figure(figsize=(20, 6))

    # Create a subplot for the non-scaled scores
    plt.subplot(1, 2, 1)
    sns.histplot(data=df, x=score_column, hue=state_column, 
                 palette={"Pathogenic": "red", "Benign": "green"}, 
                 bins=50, alpha=0.6)
    plt.title(f'Non-Scaled DMS Scores - Sample {index+1}')
    plt.xlabel('DMS Score')
    plt.ylabel('Frequency')

    # Create a subplot for the scaled scores
    plt.subplot(1, 2, 2)
    sns.histplot(data=df, x=scaled_score_column, hue=state_column, 
                 palette={"Pathogenic": "red", "Benign": "green"}, 
                 bins=50, alpha=0.6)
    plt.title(f'Scaled DMS Scores - Sample {index+1}')
    plt.xlabel('Scaled DMS Score')

    # Save the figure
    Path(output_directory)
    plt.savefig(output_directory/f'Histogram_Comparison_{index+1}.png', dpi=300)
    plt.close()

# Apply the function to every sample 
for i, df in enumerate(modified_human_DMS_samples_with_subs_list):
    plot_histogram(df, i, subs_output_directory)

for i, df in enumerate(modified_human_DMS_samples_with_indels_list):
    plot_histogram(df, i, indels_output_directory)

In [9]:
# Define function to build the automaton for the nullpeptide categories.
def build_nullpeptide_automaton(nullpeptides):
    """
    Creates an Aho - Corasick Automaton datastructure to store the nullpeptides.
    """
    
    nullpeptide_automaton = ahocorasick.Automaton() # Empty Automaton list.
    for nullpeptide in nullpeptides:
        nullpeptide_automaton.add_word(nullpeptide, nullpeptide) # Iterates through all the nullpeptides and stores each one to the automaton datastructure.
    nullpeptide_automaton.make_automaton()
    return nullpeptide_automaton

def read_and_build_automatons(nullpeptides_path, nullpeptide_lengths):
    """
    Reads nullpeptides of specific lengths and builds corresponding automatons.
    """

    automatons = {} # Initialize an empty dictionary to store automatons.

    # Loop through each specified nullpeptide length.
    for nullpeptide_length in nullpeptide_lengths:
        try:
            file_path = f"{nullpeptides_path}/nullpeptides_{nullpeptide_length}amino_acids_gencode.v43.pc_translations.txt"
            
            # Read the nullpeptide sequences from the file into a pandas DataFrame.
            nullpeptides_df = pd.read_csv(file_path)
            
            # Convert the 'nullpeptides' column of the DataFrame into a list.
            nullpeptides = nullpeptides_df['nullpeptides'].tolist()
            
            # Build an automaton for the current list of nullpeptides.
            automatons[f'automaton_{nullpeptide_length}mer'] = build_nullpeptide_automaton(nullpeptides)
        
        # Catch and report any errors that occur during the process.
        except Exception as e:
            print(f"Error processing nullpeptides of length {nullpeptide_length}: {e}")

    return automatons

nullpeptide_lengths = [4, 5, 6] # Specify nullpeptide length.

# Create the nullpeptides automatons.
automatons = read_and_build_automatons(nullpeptides_path, nullpeptide_lengths)

In [10]:
# Define functions to search the presence of each nullpeptide in the mutated sequences of the DMS experiments.
def find_nullpeptides(sequence, nullpeptide_automaton):
    """
    Searches each mutated sequence from the DMS experiments for any occurences of the nullpeptides using the previously created Automaton.
    """
    
    nullpeptide_matches = []  # Initialize an empty list to store match information
    for end_index, pattern in nullpeptide_automaton.iter(sequence):  
        # Iterate over matches found by the automaton 
        start_index = end_index - len(pattern) + 1  # Calculate start of match
        nullpeptide_matches.append((start_index, end_index, pattern)) # Store match details
    return nullpeptide_matches  # Return the list of matches

def process_DMS_sample(DMS_sample, nullpeptide_automaton, automaton_key):
    """
    Searches DMS samples for present nullpeptides in mutated sequences.
    """

    # Determine the nullpeptide length from the automaton key
    length = automaton_key.split('_')[1]

    # Column names for nullpeptides and their counts
    found_nullpeptides_col = f'{length}_Nullpeptides'
    found_nullpeptides_counts_col = f'{length}_Nullpeptides_Counts'

    # Initialize columns in DataFrame
    DMS_sample[found_nullpeptides_col] = ''
    DMS_sample[found_nullpeptides_counts_col] = 0

    # Process each row to find nullpeptides and count them
    for index, row in tqdm(DMS_sample.iterrows(), total=len(DMS_sample), desc=f"Processing for {length}"):
        matches = find_nullpeptides(row['mutated_sequence'], nullpeptide_automaton)
        if matches:
            matched_nullpeptides = [match[2] for match in matches]
            DMS_sample.at[index, found_nullpeptides_col] = ', '.join(matched_nullpeptides)
            DMS_sample.at[index, found_nullpeptides_counts_col] = len(matches)

    return DMS_sample

def thorough_nullpeptide_search(samples_list, automatons):
    """
    Creates a DataFrame for each DMS experiment category and searches for nullpeptides.
    """
    
    # Concatenate and sort the samples DataFrame
    total_samples_df = pd.concat(samples_list, ignore_index=True)
    total_samples_df.sort_values(by=['State', 'DMS_score'], ascending=[False, False], inplace=True)

    # Process for each nullpeptide category defined by automatons
    for key, automaton in automatons.items():
        total_samples_df = process_DMS_sample(total_samples_df, automaton, key)

    # Combine found nullpeptides into a single column and sum their counts
    nullpeptide_cols = [f'{key.split("_")[1]}_Nullpeptides' for key in automatons.keys()]
    total_samples_df['Nullpeptides'] = total_samples_df[nullpeptide_cols].apply(lambda row: ', '.join(filter(None, row)), axis=1).str.strip(', ')

    count_cols = [f'{key.split("_")[1]}_Nullpeptides_Counts' for key in automatons.keys()]
    total_samples_df['Nullpeptide_Counts'] = total_samples_df[count_cols].sum(axis=1)

    return total_samples_df

In [ ]:
# Process the DMS Samples
human_subs_nullp = thorough_nullpeptide_search(modified_human_DMS_samples_with_subs_list, automatons) # Substitutions
human_indels_nullp = thorough_nullpeptide_search(modified_human_DMS_samples_with_indels_list, automatons) # Indels

# Concatanate all results into a single DataFrame
human_total_nullp = pd.concat([human_subs_nullp, human_indels_nullp]).drop(columns='DMS_score')
human_total_nullp.reset_index(drop=True, inplace=True)

In [ ]:
def plot_nullpeptide_median_scores_combined(df, output_file):
    """
    Plots the median scaled DMS scores for each nullpeptide count for pathogenic and benign states,
    combined into one big plot with subplots for each study type and nullpeptide length.
    """
    study_types = df['Study_Type'].unique()
    nullpeptide_lengths = [4, 5, 6]
    states = ['Pathogenic', 'Benign']
    state_colors = {'Pathogenic': 'red', 'Benign': 'green'}

    # Create a single figure for all subplots
    fig, axes = plt.subplots(len(study_types), len(nullpeptide_lengths), figsize=(18, 6 * len(study_types)), sharey='row')

    if len(study_types) == 1:
        axes = [axes]

    for row, study_type in enumerate(study_types):
        for col, length in enumerate(nullpeptide_lengths):
            ax = axes[row][col] if len(study_types) > 1 else axes[col]
            column_name = f'{length}mer_Nullpeptides'
            
            # Filter the DataFrame to keep rows with the specified nullpeptide length and rows with zero nullpeptides
            df_length_filtered = df[(df[column_name].notna() & (df[column_name] != '')) | (df['Nullpeptide_Counts'] == 0)]
            
            for state in states:
                state_df = df_length_filtered[(df_length_filtered['Study_Type'] == study_type) & (df_length_filtered['State'] == state)]
                median_scores = state_df.groupby('Nullpeptide_Counts')['Scaled_DMS_score'].median().reset_index()

                if not median_scores.empty:
                    sns.lineplot(x='Nullpeptide_Counts', y='Scaled_DMS_score', data=median_scores, ax=ax, marker='o', linestyle='-', color=state_colors[state], label=state)
            
            ax.set_xlabel('Number of Nullpeptides')
            ax.set_ylabel('Median DMS Score')
            if col == 1:
                ax.set_title(f'{study_type}\n\n{length}mer Nullpeptides')
            else:
                ax.set_title(f'{length}mer Nullpeptides')
            
            ax.axhline(0, color='gray', linestyle='--', linewidth=0.5)
            ax.legend()

    plt.tight_layout()
    plt.savefig(output_file, dpi=300)
    plt.show()

plot_nullpeptide_median_scores_combined(human_subs_nullp, plots_directory/"subs_nullpeptide_number_score_correlation.png")
plot_nullpeptide_median_scores_combined(human_indels_nullp, plots_directory/"indels_nullpeptide_number_score_correlation.png")


In [ ]:
def plot_nullpeptide_comparison_by_study_type(df, output_file):
    """
    Plots a comparison of nullpeptide scaled scores found in the DMS samples pre study type.
    """
    study_types = df['Study_Type'].unique()
    nullpeptide_lengths = [4, 5, 6]

    group_palette = {
        'Pathogenic with Nullpeptides': '#1f77b4',  # Blue
        'Pathogenic without Nullpeptides': '#ff7f0e',  # Orange
        'Benign with Nullpeptides': '#2ca02c',  # Green
        'Benign without Nullpeptides': '#d62728'  # Red
    }
    # Create a single figure for all subplots
    fig, axes = plt.subplots(len(study_types), len(nullpeptide_lengths), figsize=(18, 6 * len(study_types)), sharey='row')

    if len(study_types) == 1:
        axes = [axes]

    for row, study_type in enumerate(study_types):
        study_df = df[df['Study_Type'] == study_type]

        for col, length in enumerate(nullpeptide_lengths):
            ax = axes[row][col] if len(study_types) > 1 else axes[col]
            column_name = f'{length}mer_Nullpeptides'

            # Separate the data by pathogenic and benign states
            pathogenic = study_df[study_df['State'] == 'Pathogenic']
            benign = study_df[study_df['State'] == 'Benign']

            # Further separate by presence of nullpeptides
            pathogenic_with_nullpeptides = pathogenic[pathogenic[column_name].notna() & (pathogenic[column_name] != '')]
            pathogenic_without_nullpeptides = pathogenic[pathogenic[column_name].isna() | (pathogenic[column_name] == '')]
            benign_with_nullpeptides = benign[benign[column_name].notna() & (benign[column_name] != '')]
            benign_without_nullpeptides = benign[benign[column_name].isna() | (benign[column_name] == '')]

            # Prepare data for plotting
            plot_data = pd.DataFrame({
                'Scaled_DMS_score': pd.concat([
                    pathogenic_with_nullpeptides['Scaled_DMS_score'], 
                    pathogenic_without_nullpeptides['Scaled_DMS_score'], 
                    benign_with_nullpeptides['Scaled_DMS_score'], 
                    benign_without_nullpeptides['Scaled_DMS_score']
                ]),
                'Group': (['Pathogenic with Nullpeptides'] * len(pathogenic_with_nullpeptides) + 
                          ['Pathogenic without Nullpeptides'] * len(pathogenic_without_nullpeptides) + 
                          ['Benign with Nullpeptides'] * len(benign_with_nullpeptides) + 
                          ['Benign without Nullpeptides'] * len(benign_without_nullpeptides))
            })

            if not plot_data.empty:
                # Plot the scores using seaborn
                sns.boxplot(x='Group', y='Scaled_DMS_score', data=plot_data, ax=ax, palette=group_palette, hue='Group')
                sns.stripplot(x='Group', y='Scaled_DMS_score', data=plot_data, ax=ax, color='black', alpha=0.5, jitter=True)
                ax.set_xlabel(' ')
                ax.set_ylabel('Scaled DMS Score')
                
                if col == 1:
                    ax.set_title(f'{study_type}\n\n{length}mer Nullpeptides')
                else:
                    ax.set_title(f'{length}mer Nullpeptides')

                # Set ticks and labels
                ax.set_xticks(range(len(plot_data['Group'].unique())))
                ax.set_xticklabels(plot_data['Group'].unique(), rotation=45, size = 14)
            else:
                if col == 1:
                    ax.set_title(f'{study_type}\n{length}mer Nullpeptides')
                else:
                    ax.set_title(f'{length}mer Nullpeptides')
                ax.set_xlabel(' ')
                ax.set_ylabel('Scaled DMS Score')
                ax.text(0.5, 0.5, 'No Data', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

    plt.tight_layout()
    plt.savefig(output_file, dpi=300)
    plt.show()

plot_nullpeptide_comparison_by_study_type(human_subs_nullp, plots_directory/"subs_scores_per_study_type.png")
plot_nullpeptide_comparison_by_study_type(human_indels_nullp, plots_directory/"indels_scores_per_study_type.png")

In [ ]:
# Define a function to plot the results as histograms so that to study the distribution of the scaled DMS scores
def plot_distributions(df_subs, df_indels, figure_path):
    """
    Plots the scaled DMS score distributions for substitutions and indels.
    """
    
    # Plotting setup
    fig, axs = plt.subplots(1, 2, figsize=(18, 6))

    # Histogram for substitutions
    sns.histplot(data=df_subs, x="Scaled_DMS_score", hue="State", palette={"Pathogenic": "red", "Benign": "green"}, bins=50, ax=axs[0], legend= False)
    axs[0].set_title("Distribution of Scaled DMS Scores for Substitutions", size=14)
    axs[0].set_xlabel("Scaled DMS Score", size=14)
    axs[0].set_ylabel("Frequency", size=14)
    axs[0].axvline(x=0, color='k', linestyle='--')  # Vertical line at x=0
    axs[0].tick_params(axis='both', which='major', labelsize=11)

    # Histogram for indels
    sns.histplot(data=df_indels, x="Scaled_DMS_score", hue="State", palette={"Pathogenic": "red", "Benign": "green"}, bins=50, ax=axs[1])
    axs[1].set_title("Distribution of Scaled DMS Scores for Indels", size=14)
    axs[1].set_xlabel("Scaled DMS Score", size=14)
    axs[1].set_ylabel(" ")
    axs[1].axvline(x=0, color='k', linestyle='--')  # Vertical line at x=0
    axs[1].tick_params(axis='both', which='major', labelsize=11)  # Adjust labelsize to your preference
    
    # Layout adjustment and saving
    plt.tight_layout()
    sns.despine()
    plt.savefig(figure_path, dpi = 300)
    plt.show()

# Plot the distributions of the scaled DMS scores for substitutions and indels.
plot_distributions(human_subs_nullp, human_indels_nullp, plots_directory/"DMS_scores_distribution.png")

In [ ]:
# Define a function to plot the results as barplots
def barplot_nullpeptides(subs_df, indels_df, nullpeptide_column, output_filename):
    """
    Creates a barplot of all 6-mer nullpeptides found in substitution and indel samples.
    """
    
    # Create a figure with two subplots side by side.
    fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)  # Share y-axis for better comparison
    
    # Prepare datasets for both substitutions and indels
    datasets = {'Substitutions': subs_df, 'Indels': indels_df}

    for idx, (label, DMS_df) in enumerate(datasets.items()):
        # Count total sequences for each state in the current dataset
        total_pathogenic_count = DMS_df[DMS_df['State'] == 'Pathogenic']['mutated_sequence'].count()
        total_benign_count = DMS_df[DMS_df['State'] == 'Benign']['mutated_sequence'].count()

        # Count sequences with at least one nullpeptide present in each state
        pathogenic_with_nullpeptide_count = DMS_df[(DMS_df['State'] == 'Pathogenic') & (DMS_df[nullpeptide_column] > 0)]['mutated_sequence'].count()
        benign_with_nullpeptide_count = DMS_df[(DMS_df['State'] == 'Benign') & (DMS_df[nullpeptide_column] > 0)]['mutated_sequence'].count()

        # Calculate proportion of counts with nullpeptides compared to total counts
        proportion_pathogenic_with_nullpeptide = pathogenic_with_nullpeptide_count / total_pathogenic_count
        proportion_benign_with_nullpeptide = benign_with_nullpeptide_count / total_benign_count

        # Create a DataFrame for plotting
        plot_data = pd.DataFrame({
            'State': ['Benign', 'Pathogenic'],
            'Proportion with Nullpeptides': [proportion_benign_with_nullpeptide, proportion_pathogenic_with_nullpeptide]
        })

        # Use seaborn to plot the bar plot on the appropriate subplot
        sns.barplot(x='State', y='Proportion with Nullpeptides', hue='State', data=plot_data, palette=['blue', 'red'], ax=axes[idx])
        axes[idx].set_title(f'{label}', fontsize=14)
        axes[idx].set_xlabel('')  # Remove X-axis label

        # Set Y-axis label only for the left subplot
        if idx == 0:
            axes[idx].set_ylabel('Proportion with Nullpeptides', fontsize=12)
        else:
            axes[idx].set_ylabel('')

        axes[idx].set_ylim(0, 1)  # Set Y-axis limits to 0 and 1
        axes[idx].grid(True, linestyle='-', alpha=0.7)  # Add grid lines

    plt.tight_layout()
    sns.despine()
    plt.savefig(output_filename, dpi=300)

    plt.show()

# Create a barplot of all 6-mer nullpeptides found in substitution and indel samples
barplot_nullpeptides(human_subs_nullp, human_indels_nullp, '6mer_Nullpeptides_Counts', plots_directory/"subs_indels_comparison_barplots.png")

In [ ]:
# Create a plot showcasing the proportions of nullpeptide counts in substitutions and indel mutations
def plot_nullpeptides_proportions(subs_df, indels_df, output_filepath):
    """
    Processes the DataFrames obtained from the Aho-Corasick algorithm for substitutions and indels,
    and plots the results as barplots that showcase the number of 6mer nullpeptides present in each state.
    """
    
    # Create a figure with two subplots
    fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
    
    # DataFrames to process: one for substitutions, one for indels
    datasets = {'Substitutions': subs_df, 'Indels': indels_df}
    column_name = '6mer_Nullpeptides_Counts'
    max_count = 5  # Define the maximum count for binning the nullpeptide counts

    for idx, (label, dataframe) in enumerate(datasets.items()):
        pathogenic = dataframe[dataframe['State'] == 'Pathogenic'][column_name]
        benign = dataframe[dataframe['State'] == 'Benign'][column_name]
        
        # Create bins with the last bin grouping all counts >= max_count
        bins = list(range(0, max_count)) + [max_count, max(pathogenic.max(), benign.max()) + 1]

        # Calculate histogram data
        hist_pathogenic, _ = np.histogram(pathogenic, bins=bins)
        hist_benign, bin_edges = np.histogram(benign, bins=bins)
        
        # Calculate proportions
        proportions_pathogenic = hist_pathogenic / hist_pathogenic.sum()
        proportions_benign = hist_benign / hist_benign.sum()

        # Create bin labels with the last label for 'max_count or more'
        bin_labels = [f"{i}" for i in bin_edges[:-2]] + [f"{max_count} or more"]
        
        # DataFrame for plotting
        plot_data = pd.DataFrame({
            'Number of Nullpeptides': bin_labels * 2,
            'Proportion': np.concatenate([proportions_pathogenic, proportions_benign]),
            'Condition': ['Pathogenic'] * len(bin_labels) + ['Benign'] * len(bin_labels)
        })

        # Plot using seaborn on the appropriate axis
        sns.barplot(x='Number of Nullpeptides', y='Proportion', hue='Condition', data=plot_data, ax=axes[idx], palette='viridis', alpha=0.8)
        axes[idx].set_title(f'{label}', fontsize=12)
        axes[idx].set_xlabel('Number of 6mer Nullpeptides', fontsize= 12)
        axes[idx].set_ylim(0, 1)
        axes[idx].set_ylabel('Proportion' if idx == 0 else '', fontsize= 12)
        tick_positions = np.arange(len(bin_labels))
        axes[idx].set_xticks(tick_positions)
        axes[idx].set_xticklabels(bin_labels, rotation=35)
        axes[idx].grid(True, linestyle='--', alpha=0.5)
        
        # Display legend only on the rightmost plot
        if idx == len(axes) - 1:
            axes[idx].legend()
        else:
            axes[idx].legend([],[], frameon=False)  # Hide legend for other plots

    plt.tight_layout()
    sns.despine()
    plt.savefig(output_filepath, dpi=300)
    plt.show()

# Plot the proportions of 6mer nullpeptides found in substitution and indel samples
plot_nullpeptides_proportions(
    subs_df=human_subs_nullp, 
    indels_df=human_indels_nullp, 
    output_filepath=plots_directory/"subs_indels_binned_nullpeptides.png"
)

In [ ]:
# Define a function to filter rows with 0 nullpeptides and also change types to improve memmory efficiency.
def filter_and_optimize_DMS_dataframe(DMS_df, remove_null_rows=False):
    """
    Filters and optimizes the DMS DataFrame for memory usage and processing speed.
    """
    
    # Filter the DataFrame to keep only rows with Nullpeptide_Counts greater than 0
    if remove_null_rows == True:
        filtered_df = DMS_df[DMS_df['Nullpeptide_Counts'] > 0]
    elif remove_null_rows == False:
        filtered_df = DMS_df.copy()
    filtered_df = filtered_df.drop(columns=['mutated_sequence', 'DMS_score', '4mer_Nullpeptides', '4mer_Nullpeptides_Counts', '5mer_Nullpeptides', '5mer_Nullpeptides_Counts', 'Nullpeptides', 'Nullpeptide_Counts'])
    filtered_df = filtered_df.rename(columns={"6mer_Nullpeptides": "Nullpeptides", "6mer_Nullpeptides_Counts": "Nullpeptide_Counts"})

    # Convert 'State' to categorical data type
    filtered_df['State'] = filtered_df['State'].astype('category')

    # Convert 'Sqrt_DMS_score' to float32
    filtered_df['Scaled_DMS_score'] = filtered_df['Scaled_DMS_score'].astype('float32')

    # Convert 'Nullpeptides' to string
    filtered_df['Nullpeptides'] = filtered_df['Nullpeptides'].astype('str')

    return filtered_df

# Apply the function and print the info for each filtered dataframe
filtered_subs_df = filter_and_optimize_DMS_dataframe(human_subs_nullp, remove_null_rows=False)
print('Substitutions dataframe info:', filtered_subs_df.info())
print(" ")

filtered_indels_df = filter_and_optimize_DMS_dataframe(human_indels_nullp, remove_null_rows=False)
print('Indels dataframe info:', filtered_indels_df.info())

In [18]:
# Define a function to encode nullpeptides in a format with counts for each state and total counts or in a format with scores for each state and total counts
def nullpeptide_encoding(ahocorasick_df, peptides_column, state_column, score_column=None, fisher_exact_results=None, encoding='count'):
    """
    Encode each nullpeptide in a format with counts for each state and total counts, or encode based on scores if specified.
    """

    # Copy dataframe to avoid modifying the original and split peptides into individual rows
    exploded_ahocorasick_df = ahocorasick_df.copy()
    exploded_ahocorasick_df[peptides_column] = exploded_ahocorasick_df[peptides_column].str.split(', ')
    exploded_ahocorasick_df = exploded_ahocorasick_df.explode(peptides_column)
    
    # Count Pathogenic and Benign occurrences for each nullpeptide
    exploded_ahocorasick_df['Pathogenic_counts'] = (exploded_ahocorasick_df[state_column] == 'Pathogenic').astype(int)
    exploded_ahocorasick_df['Benign_counts'] = (exploded_ahocorasick_df[state_column] == 'Benign').astype(int)

    if encoding == 'count':
        # Aggregate the counts for each nullpeptide
        aggregated_ahocorasick_df = exploded_ahocorasick_df.groupby(peptides_column, as_index=False).agg({
            'Pathogenic_counts': 'sum',
            'Benign_counts': 'sum'
        })

        aggregated_ahocorasick_df['Total_counts'] = aggregated_ahocorasick_df['Pathogenic_counts'] + aggregated_ahocorasick_df['Benign_counts']
        aggregated_ahocorasick_df = aggregated_ahocorasick_df.sort_values(by='Total_counts', ascending=False)
        aggregated_ahocorasick_df.reset_index(drop=True, inplace=True)
        aggregated_ahocorasick_df = aggregated_ahocorasick_df.reindex(columns=['Nullpeptides', 'Pathogenic_counts', 'Benign_counts', 'Total_counts'])

        return aggregated_ahocorasick_df

    elif encoding == 'score':
        # Initialize the score columns and aggregate scores along with counts
        exploded_ahocorasick_df['Pathogenic_scores'] = exploded_ahocorasick_df.apply(
            lambda x: [x[score_column]] if x[state_column] == 'Pathogenic' else [], axis=1)
        exploded_ahocorasick_df['Benign_scores'] = exploded_ahocorasick_df.apply(
            lambda x: [x[score_column]] if x[state_column] == 'Benign' else [], axis=1)

        aggregated_ahocorasick_df = exploded_ahocorasick_df.groupby(peptides_column, as_index=False).agg({
            'Pathogenic_counts': 'sum',
            'Benign_counts': 'sum',
            'Pathogenic_scores': lambda x: np.array([item for sublist in x for item in sublist] if x.tolist() else [0]),
            'Benign_scores': lambda x: np.array([item for sublist in x for item in sublist] if x.tolist() else [0])
        })
        aggregated_ahocorasick_df['Total_counts'] = aggregated_ahocorasick_df['Pathogenic_counts'] + aggregated_ahocorasick_df['Benign_counts']
        aggregated_ahocorasick_df['Total_counts'] = aggregated_ahocorasick_df['Total_counts'].astype(np.int16)
        aggregated_ahocorasick_df = aggregated_ahocorasick_df.reindex(columns=['Nullpeptides', 'Pathogenic_counts', 'Benign_counts', 'Total_counts', 'Pathogenic_scores', 'Benign_scores'])

        # Merge with Fisher's exact test results
        merged_ahocorasick_df = pd.merge(aggregated_ahocorasick_df, fisher_exact_results, how='left', left_on=peptides_column, right_on='Significant_Nullpeptides')
        merged_ahocorasick_df[['Odds_Ratio', 'Fisher_P_Value']] = merged_ahocorasick_df[['Odds_Ratio', 'Fisher_P_Value']].astype(np.float32)

        # Apply the lambda function to handle empty lists explicitly
        merged_ahocorasick_df['Benign_scores'] = merged_ahocorasick_df['Benign_scores'].apply(lambda x: [0] if len(x) == 0 else x)
        merged_ahocorasick_df['Pathogenic_scores'] = merged_ahocorasick_df['Pathogenic_scores'].apply(lambda x: [0] if len(x) == 0 else x)

        merged_ahocorasick_df = merged_ahocorasick_df.sort_values(by='Odds_Ratio', ascending=False)
        return merged_ahocorasick_df

    else:
        raise ValueError("Invalid encoding type specified. Choose 'count' or 'score'.")
    

# Perform statistical testing to detect nullpeptides associated with each state. Fisher exact test is used because for some nullpeptides sample size is below 5 counts.
def perform_fisher_exact_test(encoded_nullpeptides_df, test_method):
    """
    Perform Fisher Exact Test on a dataset and optionally apply a multiple testing correction.
    """

    # Create a copy of the dataframe to avoid modifying the original
    encoded_nullpeptides_df_copy = encoded_nullpeptides_df.copy()

    # Convert integer counts to int16 data type for more efficient data handling
    encoded_nullpeptides_df_copy = encoded_nullpeptides_df_copy.astype({'Pathogenic_counts': 'int16', 'Benign_counts': 'int16', 'Total_counts': 'int16'})

    # Calculate total sums once
    total_pathogenic_counts = encoded_nullpeptides_df_copy['Pathogenic_counts'].sum()
    total_benign_counts = encoded_nullpeptides_df_copy['Benign_counts'].sum()

    # Store results in a list of dictionaries
    results = []

    # Iterate over the DataFrame rows
    for _, row in encoded_nullpeptides_df_copy.iterrows():
        # Current nullpeptide counts
        current_nullepeptide_pathogenic_counts = row['Pathogenic_counts'] + 1
        current_nullepeptide_benign_counts = row['Benign_counts'] + 1
        
        # Counts for other nullpeptides (total minus current)
        other_nullpeptides_pathogenic_counts = total_pathogenic_counts - current_nullepeptide_pathogenic_counts + 1
        other_nullpeptides_benign_counts = total_benign_counts - current_nullepeptide_benign_counts + 1 
        
        # Construct the contingency table of this format
        """
        |             | Pathogenic | Benign |
        |-------------|------------|--------|
        | Nullpeptide |      x     |    y   |
        | Other       |      a     |    b   |
        """
        contingency_table = np.array([[current_nullepeptide_pathogenic_counts , current_nullepeptide_benign_counts], 
                                      [other_nullpeptides_pathogenic_counts, other_nullpeptides_benign_counts]])
        
        # Perform the Fisher Exact Test
        # The test will assess whether the odds of experiencing a specific nullpeptide being pathogenic are significantly greater compared to the rest of the nullpeptides.
        odds_ratio, p_value = stats.fisher_exact(contingency_table, test_method)

        # Append the results
        results.append({'Significant_Nullpeptides': row['Nullpeptides'], 
                        'Odds_Ratio': odds_ratio, 
                        'Fisher_P_Value': p_value})

    # Convert results to a DataFrame for easier viewing and analysis
    results_df = pd.DataFrame(results)

    # Keep only nullpeptides with adjusted p-value less than 0.05
    results_df = results_df[results_df['Fisher_P_Value'] <= 0.05]

    # Sort nullepeptides based on adjusted p-value
    results_df = results_df.sort_values(by=['Odds_Ratio', 'Fisher_P_Value'], ascending=[False, True])

    return results_df

# Define a function to filter the substitutions and indels dataframes to keep only the stastistically significant nullpeptides
def extract_nullpeptides(nullpeptide_dataframe, test_results, nullpeptide_column, fisher_test_results=True):
    """
    Filters nullpeptides in a DataFrame based on the results of the Fisher's exact test.
    """
    
    # Get the unique nullpeptides from the test results
    unique_nullpeptides = set(test_results[nullpeptide_column])

    # Helper function to process each row
    def filter_nullpeptides(peptides):
        if peptides is None:
            return None
        peptides_list = peptides.split(', ')
        filtered_significant_peptides = ', '.join([peptide for peptide in peptides_list if peptide in unique_nullpeptides])
        return filtered_significant_peptides if filtered_significant_peptides else None

    # Apply the helper function and assign to a new column
    nullpeptide_dataframe['Nullpeptides'] = nullpeptide_dataframe['Nullpeptides'].apply(filter_nullpeptides)

    # Drop rows where 'Significant_Nullpeptides' is None (i.e., no significant peptides) and drop unused original columns
    # nullpeptide_dataframe.dropna(subset=['Nullpeptides'], inplace=True)

    if fisher_test_results == True:
        nullpeptide_dataframe = nullpeptide_dataframe.drop(columns=['Nullpeptide_Counts'])
        
    return nullpeptide_dataframe

# Define a function to perform the Mann-Whitney U test to further filter nullpeptides based on scores.
def perform_mann_whitney_test(score_encoded_nullpeptides):
    """
    Applies the Mann-Whitney U test to compare 'Pathogenic_scores' and 'Benign_scores' for each row in the dataframe.
    """

    # Initialize lists to store test results
    u_stats = []
    p_values = []
    
    # Iterate over each row in the DataFrame
    for _, row in score_encoded_nullpeptides.iterrows():
        # Extract scores for pathogenic and benign
        pathogenic_scores = row['Pathogenic_scores']
        benign_scores = row['Benign_scores']
        
        # Perform Mann-Whitney U test
        u_stat, p_value = stats.mannwhitneyu(pathogenic_scores, benign_scores, alternative='two-sided')
        
        # Append results
        u_stats.append(u_stat)
        p_values.append(p_value)
    
    # Add a column containing Mann-Whitney U test p-value results
    score_encoded_nullpeptides['Mann_Whitney_P_Value'] = p_values

    # Filter results based on significance level
    filtered_score_encoded_nullpeptides = score_encoded_nullpeptides[score_encoded_nullpeptides['Mann_Whitney_P_Value'] <= 0.05]
    
    return filtered_score_encoded_nullpeptides

In [ ]:
# Define a function to process Aho-Corasick results using previously defined functions.
def detect_significant_nullpeptides_by_protein(original_dataframe):
    """
    Processes a DataFrame by splitting it by 'Protein', encoding nullpeptides, performing Fisher's exact test, and filtering significant nullpeptides.
    """
    
    # Split the DataFrame into smaller DataFrames for each unique protein
    protein_dataframes = {protein: group for protein, group in original_dataframe.groupby('Protein')}

    # Initialize a list to collect results DataFrames
    significant_nullpeptides_dfs = []
    mannwhitney_dfs = []

    # Process each smaller DataFrame
    for protein, data in tqdm(protein_dataframes.items(), desc="Processing individual proteins"):
        
        # Encode nullpeptides based on State counts for each split DataFrame
        count_encoded_df = nullpeptide_encoding(data, 'Nullpeptides', 'State', encoding='count')
        
        # Perform statistical testing on the encoded data
        fisher_results = perform_fisher_exact_test(count_encoded_df, test_method='two-sided')
        
        # Filter the original data to keep only significant nullpeptides
        fisher_significant_df = extract_nullpeptides(data, fisher_results, 'Significant_Nullpeptides', fisher_test_results=True)

        # Encode each significant nullpeptide based on the results of the Fisher's exact test, it's score and it's counts
        score_encoded_df = nullpeptide_encoding(fisher_significant_df, 'Nullpeptides', 'State', 'Scaled_DMS_score', fisher_results, encoding='score')

        # Perform Mann-Whitney U test
        mannwhitney_results = perform_mann_whitney_test(score_encoded_df)

        # Filter the dataframe to keep only significant nullpeptides found by the Mann-Whitney U test
        mannwhitney_significant_df = extract_nullpeptides(fisher_significant_df, mannwhitney_results, 'Significant_Nullpeptides', fisher_test_results=False)

        # Add a column to distinguish results from each protein
        mannwhitney_results['Protein'] = protein
        mannwhitney_significant_df['Protein'] = protein
        
        # Collect the processed DataFrame
        mannwhitney_dfs.append(mannwhitney_results)
        significant_nullpeptides_dfs.append(mannwhitney_significant_df)

        # Concatanate into a final DataFrame
        final_mannwhitney_results = pd.concat(mannwhitney_dfs)
        final_significant_df = pd.concat(significant_nullpeptides_dfs)

        # Reset the index
        final_mannwhitney_results.reset_index(drop=True, inplace=True)
        final_significant_df.reset_index(drop=True, inplace=True)

        # Drop unused columns
        final_mannwhitney_results = final_mannwhitney_results.drop(columns=['Fisher_P_Value', 'Significant_Nullpeptides'])

    # Concatenate all results into a single DataFrame
    return final_mannwhitney_results, final_significant_df

# Detect significant nullpeptides based on individual proteins for substitutions and indels
score_encoded_nullpeptides_subs, significant_nullpeptides_subs = detect_significant_nullpeptides_by_protein(filtered_subs_df)
score_encoded_nullpeptides_indels, significant_nullpeptides_indels = detect_significant_nullpeptides_by_protein(filtered_indels_df)

In [20]:
significant_nullpeptides_subs = significant_nullpeptides_subs.dropna(subset=['Nullpeptides'])
significant_nullpeptides_indels = significant_nullpeptides_indels.dropna(subset=['Nullpeptides'])

In [22]:
# Apply network analysis to find commonly co-occurring nullpeptides for each protein. Also, apply filters to keep those that are found the most times and have a significant score.
def build_nullpeptide_network(significant_nullpeptide_df, output_file, min_edge_threshold=5):
    """
    Constructs a network connecting nullpeptides that co-occur in the same DataFrame row,
    labeling edges as 'Pathogenic' or 'Benign' based on their occurrence patterns, and
    adds a score attribute for each edge.
    """

    # Initialize an empty undirected graph
    graph = nx.Graph()

    # Process each record in the DataFrame
    for _, record in significant_nullpeptide_df.iterrows():
        peptides = [peptide.strip() for peptide in record['Nullpeptides'].split(", ")]
        state = record['State']
        score = record['Scaled_DMS_score']

        # Create edges between all pairs of peptides in the row
        for i in range(len(peptides)):
            for j in range(i + 1, len(peptides)):
                peptide1, peptide2 = peptides[i], peptides[j]

                if graph.has_edge(peptide1, peptide2):
                    edge_data = graph[peptide1][peptide2]
                    edge_data['Total_count'] += 1
                    edge_data['Score_sum'] += score
                    if state == 'Pathogenic':
                        edge_data['Pathogenic_count'] += 1
                    else:
                        edge_data['Benign_count'] += 1
                else:
                    graph.add_edge(peptide1, peptide2, Total_count=1,
                                   Score_sum=score,
                                   Pathogenic_count=1 if state == 'Pathogenic' else 0,
                                   Benign_count=1 if state != 'Pathogenic' else 0)

    # Calculate mean score for each edge and remove those not meeting score threshold
    edges_to_remove = []
    for u, v, data in graph.edges(data=True):
        mean_score = data['Score_sum'] / data['Total_count']
        if abs(mean_score) < 1 or data['Total_count'] < min_edge_threshold:
            edges_to_remove.append((u, v))
        else:
            data['Mean_score'] = mean_score

    graph.remove_edges_from(edges_to_remove)

    # Label remaining edges
    for u, v, data in graph.edges(data=True):
        data['Label'] = 'Pathogenic' if data['Pathogenic_count'] > data['Benign_count'] else 'Benign'

    # Remove isolated nodes
    isolated_nodes = list(nx.isolates(graph))
    graph.remove_nodes_from(isolated_nodes)

    # Save the graph
    nx.write_graphml(graph, output_file)

    return graph

def create_adjacency_matrix(graph):
    """
    Creates and orders an adjacency matrix so that similar subnetworks are not across the diagonal.
    """

    # Extract nodes and separate them by label
    benign_nodes = [n for n, d in graph.nodes(data=True) if any(graph[u][v]['Label'] == 'Benign' for u, v in graph.edges(n))]
    pathogenic_nodes = [n for n, d in graph.nodes(data=True) if any(graph[u][v]['Label'] == 'Pathogenic' for u, v in graph.edges(n))]

    # Order nodes to emphasize grouping: benign nodes at the beginning and pathogenic at the end
    ordered_nodes = benign_nodes + pathogenic_nodes

    # Create an empty DataFrame based on the reordered nodes
    adj_matrix = pd.DataFrame(data=0, index=ordered_nodes, columns=ordered_nodes)

    # Fill the matrix with benign counts as positive and pathogenic counts as negative
    for u, v, data in graph.edges(data=True):
        value = data['Benign_count'] if data['Label'] == 'Benign' else -data['Pathogenic_count']
        adj_matrix.at[u, v] = value
        adj_matrix.at[v, u] = value  # Since the graph is undirected

    return adj_matrix

def find_top_peptides(adj_matrix, top_n=10):
    """
    Finds the top 'n' peptides in the adjacency matrix based on the absolute interaction counts.
    """
    
    # Calculate the sum of absolute interactions for each peptide
    absolute_sums = adj_matrix.abs().sum().sort_values(ascending=False)

    # Get the top 'n' peptides based on absolute interaction counts
    top_peptides = absolute_sums.head(top_n).reset_index()
    top_peptides.columns = ['Peptide', 'Counts']

    return top_peptides

In [23]:
subs_networks = {} # Dictionary to hold the networks for substitutions
indels_networks = {} # Dictionary to hold the networks for indels

subs_adjacency_matrices = {} # Dictionary to hold the adjacency matrices for substitutions
indels_adjacency_matrices = {} # Dictionary to hold the adjacency matrices for indels

subs_top_peptides = {} # Dictionary to hold the top occuring nullpeptides from substitutions for each protein 
indels_top_peptides = {} # Dictionary to hold the top occuring nullpeptides from indels for each protein

# Helper function to filter the adjacency matrix based on a threshold for occurences a nullpeptide a must possess.
def filter_adjacency_matrix(adj_matrix, threshold=100):
    """
    Filters the adjacency matrix based on a threshold for the number of occurences a nullpeptide a must possess.
    """
    
    # Mask for rows where the max absolute value is greater than the threshold
    mask = adj_matrix.map(abs).max(axis=1) > threshold
    # Filter rows and columns based on the mask
    filtered_matrix = adj_matrix.loc[mask, mask]

    return filtered_matrix

def process_nullpeptide_data(significant_nullpeptides, score_encoded_nullpeptides, mutation_type):
    """
    Creates and filters a network and its corresponding adjacency matrix showing relationships between nullpeptides per protein. 
    Also, identifies the top 10 most commonly occuring nullpeptides for each protein.
    """

    # Choose the correct dictionary for the mutation type
    if mutation_type == 'subs':
        networks_dict = subs_networks
        adj_matrices_dict = subs_adjacency_matrices
        top_peptides_dict = subs_top_peptides
        min_edge_threshold = 5

    elif mutation_type == 'indels':
        networks_dict = indels_networks
        adj_matrices_dict = indels_adjacency_matrices
        top_peptides_dict = indels_top_peptides
        min_edge_threshold = 1
    
    else:
        raise ValueError(f"Invalid mutation type: {mutation_type}")

    # Iterate over each unique protein in the DataFrame
    for protein in significant_nullpeptides['Protein'].unique():
        protein_df = significant_nullpeptides[significant_nullpeptides['Protein'] == protein]
        output_filename = f"{networks_directory}/{protein}_{mutation_type}_network.graphml"
        graph = build_nullpeptide_network(protein_df, output_filename, min_edge_threshold=min_edge_threshold)

        networks_dict[f"{protein}_network"] = graph

        # Create adjacency matrix
        adj_matrix = create_adjacency_matrix(graph)

        # Special handling for specific proteins like 'RASK'
        if protein == 'RASK':
            adj_matrix = filter_adjacency_matrix(adj_matrix)  # Specific filtering for RASK

        adj_matrices_dict[f"{protein}_adj_matrix"] = adj_matrix

        # Find top peptides based on absolute counts
        top_peptides_df = find_top_peptides(adj_matrix)
        top_peptides_dict[protein] = top_peptides_df['Peptide'].tolist()

    # Filter DataFrame to include only rows with top peptides
    most_frequent_nullpeptides = []
    score_encoded_nullpeptides_copy = score_encoded_nullpeptides.copy()

    for protein, peptides in top_peptides_dict.items():
        filtered_protein_df = score_encoded_nullpeptides_copy[
            (score_encoded_nullpeptides_copy['Protein'] == protein) & 
            (score_encoded_nullpeptides_copy['Nullpeptides'].isin(peptides))
        ]
        most_frequent_nullpeptides.append(filtered_protein_df)

    # Concatenate all filtered rows into a single DataFrame
    filtered_df = pd.concat(most_frequent_nullpeptides, ignore_index=True)

    return filtered_df

filtered_score_encoded_nullpeptides_subs = process_nullpeptide_data(significant_nullpeptides_subs, score_encoded_nullpeptides_subs, 'subs')
filtered_score_encoded_nullpeptides_indels = process_nullpeptide_data(significant_nullpeptides_indels, score_encoded_nullpeptides_indels, "indels")

In [ ]:
def plot_adjacency_matrices(adj_matrices, plot_filename, mutation_type, overall_title):
    """
    Plots multiple adjacency matrices as subplots in a single figure, skipping empty matrices.
    """

    # Filter out empty or all-zero adjacency matrices
    filtered_matrices = {k: v for k, v in adj_matrices.items() if not v.empty and not v.isnull().all().all() and not (v == 0).all().all()}

    # Determine the number of subplots needed
    n_matrices = len(filtered_matrices)
    if n_matrices == 0:
        print("No valid adjacency matrices to plot.")
        return

    n_rows = int(np.ceil(np.sqrt(n_matrices)))
    n_cols = int(np.ceil(n_matrices / n_rows))

    # Create a large figure to accommodate all subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12 * n_cols, 10 * n_rows))
    axes = axes.flatten() if n_matrices > 1 else [axes]

    for idx, (ax, (title, adj_matrix)) in enumerate(zip(axes, filtered_matrices.items())):
        simple_title = f'Protein: {title.split("_adj_matrix")[0]}'
        max_benign = adj_matrix[adj_matrix > 0].max().max()  # Max value among positive counts
        max_pathogenic = -adj_matrix[adj_matrix < 0].min().min()  # Negative of min value among negative counts

        # Handle potential NaNs if matrices are all zeros or have no valid data
        vmax = max(max_benign if np.isfinite(max_benign) else 0, max_pathogenic if np.isfinite(max_pathogenic) else 0)
        vmin = -vmax

        sns.heatmap(adj_matrix, cmap='Spectral', center=0, vmin=vmin, vmax=vmax, ax=ax, robust=True, cbar=(mutation_type == 'indels' or (mutation_type == 'subs' and idx % n_cols == 2)))

        if mutation_type == 'subs' and idx % n_cols == 2:
            cbar = ax.collections[0].colorbar
            cbar.set_ticks([vmin, 0, vmax])
            cbar.set_ticklabels([f'Pathogenic\ncounts', '0', f'Benign\ncounts'], size=16)
        elif mutation_type == 'indels':
            cbar = ax.collections[0].colorbar
            if cbar is not None:
                cbar.set_ticks([vmin, 0, vmax])
                cbar.set_ticklabels([f'Pathogenic\ncounts', '0', f'Benign\ncounts'], size=16)

        ax.set_yticklabels(ax.get_yticklabels(), rotation=0)  # Rotate y-ticks for better visibility
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)  # Rotate x-ticks for better visibility
        ax.set_title(simple_title, size=22)

        # Set aspect ratio to equal
        ax.set_aspect('equal')

    # Hide any unused axes if there are any
    for ax in axes[len(filtered_matrices):]:
        ax.axis('off')
    if mutation_type == 'subs':
        plt.suptitle(overall_title, size=52, weight='bold')
    elif mutation_type == 'indels':
        plt.suptitle(overall_title, size=22, weight='bold')

    plt.tight_layout(rect=[0, 0, 0.99, 0.98])  # Adjust layout to accommodate the title
    sns.despine()
    plt.savefig(plot_filename, dpi=300)
    plt.show()

# Plot the adjacency matrices for each protein
plot_adjacency_matrices(subs_adjacency_matrices,
                        plot_filename= plots_directory/"subs_adjacency_matrices.png",
                        mutation_type='subs',
                        overall_title='Substitutions')

plot_adjacency_matrices(indels_adjacency_matrices,
                        plot_filename=plots_directory/"indels_adjacency_matrices.png",
                        mutation_type='indels',
                        overall_title='Indels')


In [ ]:
def plot_peptides_scores_by_protein(score_encoded_dataframe, output_file, mutation_type, title):
    """
    Creates boxplots per protein showcasing the distribution of pathogenic and benign scores for the top 10 most frequent nullpeptides.
    """
    if len(score_encoded_dataframe) == 0:
        print('No scores to plot.')
        return
    
    proteins = score_encoded_dataframe['Protein'].unique()
    valid_datasets = len(proteins)  # The number of unique proteins determines the number of datasets

    # Dynamically calculate rows based on the number of datasets
    if mutation_type == 'subs':
        cols = 3  # Define the number of columns in the subplot grid
    elif mutation_type == 'indels':
        cols = 1  # Define the number of columns in the subplot grid

    rows = (valid_datasets + cols - 1) // cols  # Calculate the required number of rows

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 6, rows * 4))

    # If there's only one subplot, wrap axes in a list to simplify the loop
    if rows * cols == 1:
        axes = [axes]
    else:
        axes = axes.flatten()  # Flatten the array to simplify the loop

    # Loop through each protein and create a plot
    for i, (ax, protein) in enumerate(zip(axes, proteins)):
        data = score_encoded_dataframe[score_encoded_dataframe['Protein'] == protein]
        melted_data = data.melt(id_vars=['Nullpeptides'], value_vars=['Pathogenic_scores', 'Benign_scores'],
                                var_name='Type', value_name='Score')
        melted_data = melted_data.explode('Score')
        melted_data['Score'] = melted_data['Score'].astype(float)
        
        # Replace the type values with 'Pathogenic' and 'Benign'
        melted_data['Type'] = melted_data['Type'].replace({'Pathogenic_scores': 'Pathogenic', 'Benign_scores': 'Benign'})

        # Create boxplot and stripplot
        bp = sns.boxplot(x='Nullpeptides', y='Score', hue='Type', data=melted_data, palette='viridis', width=0.7, boxprops=dict(alpha=.8), fliersize=0, ax=ax)
        sns.stripplot(x='Nullpeptides', y='Score', hue='Type', data=melted_data, palette='dark:k', size=3, jitter=True, dodge=True, alpha=0.5, ax=ax)

        # Only show y-axis label on the first plot of each row
        if i % cols == 0:
            ax.set_ylabel('Protein Fitness', size=12)
        else:
            ax.set_ylabel('')

        ax.set_xlabel('')

        # Set the y-ticks with two decimal points
        ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
        if protein == 'YAP1':
            ax.yaxis.set_major_locator(mticker.MultipleLocator(2.0))
        else:
            ax.yaxis.set_major_locator(mticker.MultipleLocator(0.5))

        # Rotate the x-ticks for better readability using tick_params
        ax.tick_params(axis='x', rotation=25)

        # Remove legends from all but the top right plot
        ax.get_legend().remove()

        ax.set_title(f'Protein: {protein}')
    
    if mutation_type == 'subs':
        fig.suptitle(title, size=22, weight='bold')
    if mutation_type == 'indels':
        fig.suptitle(title, size=12, weight='bold')

    # If there are fewer plots than axes, turn off the extra axes
    for j in range(valid_datasets, rows * cols):
        axes[j].axis('off')

    plt.tight_layout(rect=[0, 0, 0.9, 1])  # Adjust layout to accommodate the legend
    plt.savefig(output_file, dpi=300, bbox_inches='tight')  # Save with tight bounding box
    plt.show()

# Plot boxplots for the top 10 most frequent peptides by protein
plot_peptides_scores_by_protein(filtered_score_encoded_nullpeptides_subs, 
                                plots_directory/"subs_box_plots.png", 
                                mutation_type='subs',
                                title='Substitutions')

plot_peptides_scores_by_protein(filtered_score_encoded_nullpeptides_indels, 
                                plots_directory/"indels_box_plots.png", 
                                mutation_type='indels',
                                title=' ')

# Since NKX31 is possesses only one nullpeptide, we plot a separate boxplot, otherwise the result would be lost
NKX31_data = score_encoded_nullpeptides_indels[score_encoded_nullpeptides_indels['Protein'] == 'NKX31']
plot_peptides_scores_by_protein(NKX31_data,
                                plots_directory/"indels_NKX31_box_plots.png",
                                mutation_type='indels',
                                title='Indels')

In [26]:
# Count encode and plot the original unfiltered nullpeptides per protein to study the original distribution of pathogenic and benign nullpeptides
def count_encode_nullpeptides_by_protein(original_dataframe):
    """
    Processes a DataFrame by splitting it by 'Protein' and encoding nullpeptides based on their 'State' counts.
    """

    # Split the DataFrame into smaller DataFrames for each unique protein
    protein_dataframes = {protein: group for protein, group in original_dataframe.groupby('Protein')}

    # Initialize a list to collect results DataFrames
    encoded_dfs = []

    # Process each smaller DataFrame
    for protein, data in protein_dataframes.items():
        # Encode nullpeptides based on State counts for each split DataFrame
        count_encoded_df = nullpeptide_encoding(data, 'Nullpeptides', 'State', encoding='count')
        count_encoded_df['Protein'] = protein  # Add protein name to the DataFrame
        encoded_dfs.append(count_encoded_df)

    # Concatenate all results into a single DataFrame and reset the index
    final_encoded_df = pd.concat(encoded_dfs).reset_index(drop=True)

    return final_encoded_df

# Count encode nullpeptides per protein
count_encoded_nullpeptides_subs = count_encode_nullpeptides_by_protein(filtered_subs_df)
count_encoded_nullpeptides_indels = count_encode_nullpeptides_by_protein(filtered_indels_df)

In [ ]:
def plot_aggregated_proportions(aggregated_data, mutation_type):
    """
    Plots the stacked bar chart for the given aggregated data, sorting by pathogenic proportion.
    """

    # Sort the data by Pathogenic proportion in descending order
    aggregated_sorted = aggregated_data.sort_values(by='Pathogenic_proportion', ascending=True)

    # Create a figure and one subplot
    fig, ax = plt.subplots(figsize=(10, 18))
    colors = sns.color_palette("viridis")  # Set a color palette for the bars

    # Plot horizontal stacked bars
    bar_width = 0.8
    ax.barh(aggregated_sorted['Protein'], aggregated_sorted['Pathogenic_proportion'], color=colors[2], label='Pathogenic', alpha=0.85, height=bar_width)
    ax.barh(aggregated_sorted['Protein'], aggregated_sorted['Benign_proportion'], left=aggregated_sorted['Pathogenic_proportion'], color=colors[3], label='Benign', alpha=0.85, height=bar_width)

    ax.set_yticks(range(len(aggregated_sorted)))
    ax.set_yticklabels(aggregated_sorted['Protein'], fontsize=12)
    ax.tick_params(axis='x', labelsize=12)
    ax.set_xlabel('Proportion of Pathogenic and Benign Nullpeptides', size=14)
    ax.axvline(x=0.5, color='black', linewidth=2, linestyle='dotted', alpha=0.6)
    ax.set_xlim(0, 1)
    
    plt.tight_layout()

    # Adjust alignment
    for label in ax.get_yticklabels():
        label.set_verticalalignment('center')

    # Create a legend outside the plot area for 'subs' mutation type
    if mutation_type == 'subs':
        ax.set_title('Substitutions', size=34, weight='bold')

    if mutation_type == 'indels':
        legend = fig.legend(loc='upper right', bbox_to_anchor=(1.22, 0.05), bbox_transform=ax.transAxes,
                            prop={'size': 14}, frameon=True, framealpha=0.9, fancybox=True)
        ax.set_title('Indels', size=34, weight='bold')
    
    plt.savefig(f"{plots_directory}/{mutation_type}_original_stacked_bar_plot.png", dpi=300, bbox_inches='tight')
    plt.show()

# Aggregate and calculate proportions for Indels
indel_aggregated = count_encoded_nullpeptides_indels.groupby('Protein').agg({
    'Pathogenic_counts': 'sum',
    'Benign_counts': 'sum'
}).reset_index()
indel_aggregated['Total'] = indel_aggregated['Pathogenic_counts'] + indel_aggregated['Benign_counts']
indel_aggregated['Pathogenic_proportion'] = indel_aggregated['Pathogenic_counts'] / indel_aggregated['Total']
indel_aggregated['Benign_proportion'] = indel_aggregated['Benign_counts'] / indel_aggregated['Total']

# Aggregate and calculate proportions for Substitutions
substitution_aggregated = count_encoded_nullpeptides_subs.groupby('Protein').agg({
    'Pathogenic_counts': 'sum',
    'Benign_counts': 'sum'
}).reset_index()
substitution_aggregated['Total'] = substitution_aggregated['Pathogenic_counts'] + substitution_aggregated['Benign_counts']
substitution_aggregated['Pathogenic_proportion'] = substitution_aggregated['Pathogenic_counts'] / substitution_aggregated['Total']
substitution_aggregated['Benign_proportion'] = substitution_aggregated['Benign_counts'] / substitution_aggregated['Total']

# Plot for Substitutions and Indels
plot_aggregated_proportions(substitution_aggregated, 'subs')
plot_aggregated_proportions(indel_aggregated, 'indels')